In [1]:
import pandas as pd
from src.recommenders import ImprovedRecommender, PopBasedRecommender
from sklearn.decomposition import TruncatedSVD, PCA, IncrementalPCA, KernelPCA, SparsePCA
import numpy as np
import pickle

In [2]:
output_path = "./evaluation/"
items_path = "./data/games.pkl.gz"
data_path = "./data/interactions_splits_"
reviews_path = "./data/reviews.parquet"

In [13]:
import re
def parse_json(filename_python_json: str, read_max: int = -1) -> pd.DataFrame:
    """Parses json file into a DataFrame

    Args:
        filename_python_json (str): Path to json file
        read_max (int, optional): Max amount of lines to read from json file. Defaults to -1.

    Returns:
        DataFrame: DataFrame from parsed json
    """
    with open(filename_python_json, "r", encoding="utf-8") as f:
        # parse json
        parse_data = []
        # tqdm is for showing progress bar, always good when processing large amounts of data
        for line in f:
            # load python nested datastructure
            parsed_result = eval(line)
            parse_data.append(parsed_result)
            if read_max != -1 and len(parse_data) > read_max:
                print(f"Break reading after {read_max} records")
                break
        print(f"Reading {len(parse_data)} rows.")

        # create dataframe
        df = pd.DataFrame.from_dict(parse_data)
        return df
    
games = parse_json("./data/raw/steam_games.json")
games.drop(["url", "reviews_url", "discount_price", "title", "release_date"], axis=1, inplace=True)

games.to_pickle("./data/all_games.pkl.gz")
games

Reading 32135 rows.


,publisher,genres,app_name,tags,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,"[Action, Adventure, Casual]",[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",NaN,NaN
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,NaN,NaN
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,NaN,NaN
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns",1 user reviews,NaN


In [3]:
evaluations = list()
use_data = ["specs", "genres", "tags", "early_access", "publisher"]
weighting_scheme = {"playtime": True, "sentiment": "mixed", "reviews": True}

rec = ImprovedRecommender(items_path, train_path=f"{data_path}0_train.parquet", test_path=f"{data_path}0_test.parquet",
                            val_path=f"{data_path}0_val.parquet", reviews_path=reviews_path, sparse=True, tfidf="smooth", normalize=True, dim_red=None, columns=use_data, weighting_scheme=weighting_scheme)
rec.generate_recommendations(amount=20, silence=True, read_max=20)

rec.qualitative_evaluation()

,owned_items,recommended_items
35844,"[Uplink, Call of Duty: World at War, DEFCON, M...","[Operation Flashpoint: Red River, Red Orchestr..."
12778,"[Natural Selection 2, Dragon Age: Origins - Ul...","[Worms W.M.D, Mount & Blade, Knights of Pen an..."
52734,[The Expendabros],"[Broforce, Rocketbirds: Hardboiled Chicken, Fo..."
27853,"[Operation Flashpoint: Dragon Rising, Colin Mc...","[F1 2016, Tropico 5, Planet Coaster, F1 2014, ..."
45757,"[Just Cause, Guns of Icarus Online, Poker Nigh...","[Mount & Blade: Warband, Fallout 3, Saints Row..."
36398,"[Double Action: Boogaloo, Poker Night 2, Natur...","[Copa Petrobras de Marcas, Codename CURE, BLOC..."
561,[BIT.TRIP Presents... Runner2: Future Legend o...,"[The Escapists, METAL SLUG 3, Waveform, Polari..."
9562,"[Hacker Evolution, ""Glow Ball"" - The billiard ...","[RIFT, Mabinogi, Riders of Icarus, Dragons and..."
52541,[Eternal Senia],"[LiEat, Skyborn, Helen's Mysterious Castle, Th..."
37819,"[Bloons TD Battles, Sven Co-op, Boson X, Tacti...","[Tales from the Borderlands, The Walking Dead,..."
